# Faces scenario

This notebook creates a new scenario with the face of the person in the video masked in.

## Create a new scenario

In [ ]:
import respiration.roi as roi


def get_face_masks(frames):
    """This function returns the segmentation masks of the face of the person in the video."""
    masks = []
    for frame in frames:
        faces = roi.detect_faces(frame)
        roi_mask = np.zeros_like(frame)

        for face in faces:
            mask = roi.mask_from_area(frame, face)
            roi_mask = np.maximum(roi_mask, mask)

        masks.append(roi_mask)
    return masks

In [ ]:
import cv2
import numpy as np


def store_video_with_mask(frames, masks, filename, fps):
    shape = frames[0].shape

    fourcc = cv2.VideoWriter_fourcc(*'FFV1')
    out = cv2.VideoWriter(filename, fourcc, fps, (shape[1], shape[0]))

    for idx, mask in enumerate(masks):
        frame = np.uint8(frames[idx] * mask)
        out.write(frame)

    out.release()

In [ ]:
import os
import subprocess
from respiration.dataset import VitalCamSet
from tqdm.auto import tqdm

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])

for (subject, setting) in tqdm(scenarios):
    print(f"subject: {subject}")

    destination = os.path.join(dataset.data_path, subject, '302_face_only')

    # Copy the unisens data to the new scenario
    source = os.path.join(dataset.data_path, subject, setting, 'synced_Logitech HD Pro Webcam C920')
    os.makedirs(destination, exist_ok=True)
    subprocess.run(["cp", "-r", source, destination])

    frames, meta = dataset.get_video_rgb(subject, setting)
    masks = get_face_masks(frames)
    video_path = os.path.join(destination, 'Logitech HD Pro Webcam C920.avi')
    store_video_with_mask(frames, masks, video_path, meta.fps)